In [ ]:
import numpy
import pandas
from src.data_objects import AllData
from src.functions import *

retrain = True
runchain = True

model_par = "input/epemDesign.txt"
ThisData = AllData
outdir = "/data/rjfgroup/rjf01/cameron.parker/tunes/epem/"

Trimming points and obs ranges

In [ ]:
trimRanges(ThisData)

In [ ]:
badpoints = []
trimPoints(badpoints,ThisData)

Making Data pkl for selected observables

In [ ]:
buildDataPkl(ThisData)
print(ThisData["datapkl"])

Getting emulators

In [ ]:
from src.emulator_BAND import EmulatorBAND

if retrain:
    buildObsPkls(ThisData)
    trainEmulators(model_par, ThisData)
else:
    readEmulators(ThisData)

Running Chain

In [ ]:
from src.mcmc import Chain
import os

mymcmc = Chain(expdata_path=ThisData["datapkl"], model_parafile=model_par)
mymcmc.loadEmulator(getEmuPathList(ThisData))

In [ ]:
os.environ["OMP_NUM_THREADS"] = "1"
# may have to: export RDMAV_FORK_SAFE=1 before running the code

n_effective=2000
n_active=1000
n_prior=4000
sample="tpcn"
n_max_steps=100
random_state=42

n_total = 25000
n_evidence = 0

pool = 12

if runchain:
    sampler = mymcmc.run_pocoMC(n_effective=n_effective, n_active=n_active,
                            n_prior=n_prior, sample=sample,
                            n_max_steps=n_max_steps, random_state=random_state,
                            n_total=n_total, n_evidence=n_evidence, pool=pool)

Corner Plot

In [ ]:
import pickle
import corner
import matplotlib.pyplot as plt
import numpy as np
        
with open('mcmc/chain.pkl', 'rb') as pf:
        data = pickle.load(pf)

labels = mymcmc.label

fig = corner.corner(data['chain'], weights=data['weights'], labels=labels, color="C0")
plt.show()

In [ ]:
TransformedSamples = np.copy(data['chain'])
TransformedSamples[:,0] = data['chain'][:,0]
TransformedSamples[:,1] = (2*data['chain'][:,3]+0.05) + (data['chain'][:,0]-(2*data['chain'][:,3]+0.05))*data['chain'][:,1]
TransformedSamples[:,2] = data['chain'][:,2]
TransformedSamples[:,3] = data['chain'][:,3]
TransformedSamples[:,4] = data['chain'][:,4]
TransformedSamples[:,5] = data['chain'][:,5]

labels[1] = "QS"
fig = corner.corner(TransformedSamples, weights=data['weights'], labels=labels, color="C0")
plt.show()

bestParams = extract_parameters(TransformedSamples, labels, "/data/rjfgroup/rjf01/cameron.parker/tunes/epem/")

Priors

In [ ]:
makeplot(ThisData, "Priors", "/data/rjfgroup/rjf01/cameron.parker/tunes/epem/")

Posteriors

In [ ]:
makeplot(ThisData, "Posteriors", "/data/rjfgroup/rjf01/cameron.parker/tunes/epem/", samples=data["chain"])